In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision import models

from tqdm.notebook import tqdm
from pathlib import Path

In [2]:
DATA_DIR = Path("/kaggle/input/chest-xray-pneumonia/chest_xray/")

class ChestXRayDataset(Dataset):
    def __init__(self, source_dir, transformer = None):
        self.data = ImageFolder(source_dir, transform=transformer)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    
    @property
    def classes(self):
        return self.data.classes


In [3]:
transformer = transforms.Compose(
    [
        transforms.Resize(size=(256,256)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

train_data = ChestXRayDataset(DATA_DIR / "train", transformer)
test_data = ChestXRayDataset(DATA_DIR / "test", transformer)
val_data = ChestXRayDataset(DATA_DIR / "val", transformer)

In [4]:
batch_size = 256
train_loader = DataLoader(train_data, batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size, shuffle=False)
val_loader = DataLoader(val_data, batch_size, shuffle=False)

In [5]:
class Net(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.base_model = models.efficientnet_b0(
            weights=models.EfficientNet_B0_Weights.DEFAULT
        )
        self.layer = nn.Sequential(nn.Flatten(), nn.Linear(1000, 2))

    def forward(self, x):
        x = self.base_model(x)
        return self.layer(x)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net()
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 143MB/s]


Using 2 GPUs!


In [7]:
epochs = 50
model.train()
for epoch in tqdm(range(epochs), desc="EPOCH"):
    for images, labels in tqdm(train_loader, leave=False):
        images, labels = images.to(device), labels.to(device)

        output = model(images)
        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"EPOCH: {epoch}  LOSS: {loss.item()}")

    del images, labels
    torch.cuda.empty_cache()

EPOCH:   0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

EPOCH: 0  LOSS: 0.42538225650787354


  0%|          | 0/21 [00:00<?, ?it/s]

EPOCH: 1  LOSS: 0.15832731127738953


...

EPOCH: 47  LOSS: 0.036510538309812546


  0%|          | 0/21 [00:00<?, ?it/s]

EPOCH: 48  LOSS: 0.0020056909415870905


  0%|          | 0/21 [00:00<?, ?it/s]

EPOCH: 49  LOSS: 3.621662835939787e-05


In [16]:
correct, total = 0, 0
train_loss = 0
model.eval()
with torch.no_grad():
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        output = model(images)
        prediction = torch.argmax(output.data, 1)
        total += labels.size(0)
        correct += (prediction == labels).sum().item()
        
        train_loss += loss.item()

    del images, labels
    torch.cuda.empty_cache()
print("Train Accuracy:", 100*correct/total)

  0%|          | 0/21 [00:00<?, ?it/s]

Train Accuracy: 99.96165644171779


In [14]:
correct, total = 0, 0
val_loss = 0
model.eval()
with torch.no_grad():
    for images, labels in tqdm(val_loader):
        images, labels = images.to(device), labels.to(device)

        output = model(images)
        prediction = torch.argmax(output.data, 1)
        total += labels.size(0)
        correct += (prediction == labels).sum().item()
        
        val_loss += loss.item()

    del images, labels
    torch.cuda.empty_cache()
print("Validation Accuracy:", 100*correct/total)

  0%|          | 0/1 [00:00<?, ?it/s]

Validation Accuracy: 93.75


In [15]:
correct, total = 0, 0
test_loss = 0
model.eval()
with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images, labels = images.to(device), labels.to(device)

        output = model(images)
        prediction = torch.argmax(output.data, 1)
        total += labels.size(0)
        correct += (prediction == labels).sum().item()
        
        test_loss += loss.item()

    del images, labels
    torch.cuda.empty_cache()
print("Test Accuracy:", 100*correct/total)

  0%|          | 0/3 [00:00<?, ?it/s]

Test Accuracy: 81.08974358974359


In [17]:
torch.save({
    "epochs": epochs,
    "model": model,
    "state_dict": model.state_dict(),
    "optimizer": optimizer.state_dict()
},
"PneumoniaClassificationModel.pth")

torch.save(model.state_dict(), "PneumoniaDetector.pth")
torch.save(model, "PneumoniaClassifier.pth")

In [ ]:
print("Model Saved!")

### Load Model

In [18]:
model_data = torch.load("/kaggle/working/PneumoniaClassificationModel.pth")

classifier = model_data["model"]
classifier.load_state_dict(model_data["state_dict"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


classifier.to(device)
print("Done")

Done


In [ ]:
correct, total = 0, 0
test_loss = 0
criterion = nn.CrossEntropyLoss()
classifier.eval()
with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images, labels = images.to(device), labels.to(device)

        output = classifier(images)
        loss = criterion(output, labels)
        prediction = torch.argmax(output.data, 1)
        total += labels.size(0)
        correct += (prediction == labels).sum().item()
        
        test_loss += loss.item()

    del images, labels
    torch.cuda.empty_cache()
print("Test Accuracy:", 100*correct/total)

In [20]:
# Test
classes = ["NORMAL", "PNEUMONIA"]
from PIL import Image
image_path = "/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person100_bacteria_475.jpeg"
image = Image.open(image_path).convert("RGB")

transformer = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])


transformer(image).shape

with torch.no_grad():    
    image_tensor = transformer(image)
    image_tensor = torch.unsqueeze(image_tensor, 0)
    image_tensor.shape
    output = classifier(image_tensor)
    prediction = torch.argmax(output.data, 1)
    print(classes[prediction.item()])

PNEUMONIA
